In [1]:
import numpy as np
import pandas as pd

In [2]:
# hostm = pd.read_excel("data/Host_metabolome.xlsx", sheet_name=None)

In [3]:
biom = pd.read_excel("data/meta-metabolome.xlsx", sheet_name=None)

In [4]:
labels = pd.read_excel("data/Microbiome (1).xlsx", sheet_name=None)

In [5]:
enum_times = [i for i in biom.keys()]

nature of potential data errors

###### stats

In [6]:
for t in enum_times:
    s = len(biom[t])
    f = 0
    fi = 0
    for i in range(s):
        curr = []
        for j in ['BR1', 'BR2', 'BR3']:
            if not np.isnan(biom[t][j][i]):
                curr.append(biom[t][j][i])
        if len(curr) < 3:
            f += 1
        cv = np.std(curr)/np.mean(curr)
        if cv > 0.5:
            fi += 1
    print("{}: nan values {}, too wide {}".format(t, f*100/s, fi*100/s))

3wk: nan values 53.160919540229884, too wide 33.91909814323608
4wkTC: nan values 49.58089332874038, too wide 26.145366861867036
4wkAB: nan values 43.582807333934475, too wide 26.59052199178439
6wkTC: nan values 58.15899581589958, too wide 32.75009509319133
6wkAB: nan values 50.84089894329513, too wide 27.429677035273105
8wkTC: nan values 54.90307867730901, too wide 35.034207525655646
8wkAB: nan values 44.47420634920635, too wide 27.36111111111111
10wkTC: nan values 44.4593895499224, too wide 28.20486290739783
10wkAB: nan values 55.44083684282027, too wide 32.237467735362046


In [7]:
for j in ['BR1', 'BR2', 'BR3']:
    e = []
    for t in enum_times:
        s = len(biom[t])
        f = 0
        for i in range(s):
            if np.isnan(biom[t][j][i]):
                f += 1
        e.append(f*100/s)
    print("mean nan amount for {}: {}".format(j, np.mean(e)))        

mean nan amount for BR1: 18.213078070565878
mean nan amount for BR2: 16.376306943138655
mean nan amount for BR3: 15.921951250890743


take `BR3` for now.

In [8]:
m3biom = biom.copy()
for t in enum_times:
    m3biom[t] = biom[t].copy(deep=True)
    m3biom[t].drop(columns= ['BR1', 'BR2'], inplace=True)
    m3biom[t].rename(columns = {'BR3':'value'}, inplace = True)
    m3biom[t].dropna(inplace=True)


label data

assuming mutually exclusive

In [9]:
m3l1 = {}
for t in enum_times:
    m3l1[t] = labels['Phylum'][['index', t+'3']]
    m3l1[t] = m3l1[t][~(m3l1[t] == 0.0).any(axis=1)]

In [10]:
m3l2 = {}
for t in enum_times:
    m3l2[t] = labels['Genus'][['index', t+'3']]
    m3l2[t] = m3l2[t][~(m3l2[t] == 0.0).any(axis=1)]

In [11]:
m3l = {}
for t in enum_times:
    m3l[t] = pd.concat([m3l1[t], m3l2[t]])
